# Regresion Lineal Múltiple

# Maestría en Ciencia de Datos

## Francisco ME

In [ ]:
# Cargar librerias y dependencias
import numpy as np
import pandas as pd
import statsmodels.api as sm
import visualizaciones as vs
from sklearn import linear_model
from datos import pd_hist as precios

In [ ]:
# para visualizar data frames sin restriccion
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 100)

In [ ]:
# visualizar datos de entrada
